# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

В рамках проекта я исследую месторождения нефти. Передо мной стоит задача найти лучшее месторождение, вероятность прибыли в котором будет выше остальных. Для этого мне предстоит строить модели линейной регрессии, а также использовать иные методы для предсказания результата и максимизации потенциальной прибыли. 

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
!pip install phik
from sklearn.utils import shuffle
import phik
from phik.report import plot_correlation_matrix
from phik import report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from scipy import stats as st
from sklearn.utils import resample
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import random

In [3]:
df1 = pd.read_csv('geo_data_0.csv') 
df2 = pd.read_csv('geo_data_1.csv')
df3 = pd.read_csv('geo_data_2.csv')
display(df1.sample(5)) 
display(df2.sample(5))
display(df3.sample(5))

,id,f0,f1,f2,product
38359,bCP23,1.889713,-0.185329,1.043781,79.216200
84566,sSXir,0.225669,1.047047,-1.537502,30.527584
46739,h1Aho,1.485535,-0.227568,9.704289,138.039593
43784,StCBx,0.290682,0.990295,3.814065,141.936431
25094,HDM0X,0.149577,1.013807,1.150723,41.376682


,id,f0,f1,f2,product
11332,z1yGQ,3.627591,-2.950377,2.000484,53.906522
47792,hmpH9,-3.225782,-6.733122,1.004068,30.132364
35269,y4PbH,-5.066039,-11.998665,2.014730,57.085625
63330,B1RuH,8.896351,-0.742083,4.003318,107.813044
46257,DtFiu,14.341562,-2.163920,3.001264,80.859783


,id,f0,f1,f2,product
80756,tVHGC,3.403742,1.391011,9.995557,158.644451
18055,Hm3eB,-0.484726,2.273583,4.155130,111.232748
33800,R2F00,-1.732737,0.360808,2.487267,29.662522
67208,tw3JZ,3.037952,-0.756916,5.589815,105.351481
37531,J6fpT,1.126733,-0.242567,8.983896,137.896023


In [4]:
df1.info()
df2.info() 
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [5]:
display(df1.isna().sum())
display(df2.isna().sum())
display(df3.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Проверяем датафреймы на признак пропусков. 

In [6]:
corr1 = df1.corr()
corr1.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [7]:
corr2 = df2.corr()
corr2.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


На выводе матрицы корреляций видно, что признак f2 сильно коррелирует с переменной product. Это говорит о том, что может возникнуть проблема мультиколлинеарности. Поскольку такая проблема встречается только в одном из датафреймов, столбец останется без изменений. 

In [8]:
corr3 = df3.corr() 
corr3.style.background_gradient(cmap='coolwarm')

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


## Обучение и проверка модели

In [9]:
warnings.filterwarnings('ignore')

features1 = df1.drop(['product'], axis=1)
target1 = df1['product']
features1 = features1.drop(columns='id') 

features2 = df2.drop(['product'], axis=1)
target2 = df2['product']
features2 = features2.drop(columns='id') 

features3 = df3.drop(['product'], axis=1)
target3 = df3['product']
features3 = features3.drop(columns='id') 

features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1 , target1, test_size=0.25, random_state=12345)

features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2 , target2, test_size=0.25, random_state=12345)

features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    features3 , target3, test_size=0.25, random_state=12345)



Создадим features и target согласно заданию. 

In [10]:
model1 = LinearRegression(n_jobs=1000)
model1.fit(features_train1, target_train1)
predictions1 = model1.predict(features_valid1) 
score = model1.score(features_valid1, target_valid1)
rmse = (mean_squared_error(target_valid1, predictions1)) ** 0.5
print('Оценка модели равна:', score)
print('Средний запас равен:', predictions1.mean())
print('RMSE равно:', rmse)


Оценка модели равна: 0.27994321524487786
Средний запас равен: 92.59256778438035
RMSE равно: 37.5794217150813


Проверим модель на выборке первого региона. 

In [11]:
model2 = LinearRegression(n_jobs=1000)
model2.fit(features_train2, target_train2)
predictions2 = model2.predict(features_valid2) 
score = model2.score(features_valid2, target_valid2)
rmse = (mean_squared_error(target_valid2, predictions2)) ** 0.5
print('Оценка модели равна:', score)
print('Средний запас равен:', predictions2.mean())
print('RMSE равно:', rmse)


Оценка модели равна: 0.9996233978805127
Средний запас равен: 68.728546895446
RMSE равно: 0.8930992867756182


На выборке второго региона

In [12]:
model3 = LinearRegression()
model3.fit(features_train3, target_train3)
predictions3 = model3.predict(features_valid3) 
score = model3.score(features_valid3, target_valid3)
rmse = (mean_squared_error(target_valid3, predictions3)) ** 0.5
print('Оценка модели равна:', score)
print('Средний запас равен:', predictions3.mean())
print('RMSE равно:', rmse)

Оценка модели равна: 0.20524758386040465
Средний запас равен: 94.96504596800489
RMSE равно: 40.02970873393434


И на выборке третьего региона. 

Таким образом можно заметить, что самый воский коэффициент точности у второй модели -- 0.99, а самый низкий -- у третьей, 0.2. Таким образом можно сказать, что в среднем запасы скважин в регионах равны 92, 68, 94 соответственно. Но при наименьшем RMSE средние запасы у второго региона составляют 68 единиц. Таким образом надо выбрать регион, в котором средние запасы выше, чтобы максимизировать вероятность прибыли и минимизировать риски. 

## Подготовка к расчёту прибыли

In [13]:
BUDGET = 10000000000
PROFIT_PER_BAR = 450000
min_profit = (BUDGET / PROFIT_PER_BAR) / 200
print('Минимальные запасы для безубыточной разработки:', min_profit)

Минимальные запасы для безубыточной разработки: 111.11111111111111


Таким образом можно сказать, что при значениях, показанных моделью, бизнес не будет окупаться, потому что минимальное количество запасов будет меньше точки безубыточности. 

Подготовим переменные для того, чтобы использовать их в дальнейшем расчете. 
Также стоит учитывать, что каждая единица продукции будет стоить 450 000 руб., это будет использования в расчетах далее. 

## Расчёт прибыли и рисков 

In [14]:
target_valid1 = target_valid1.set_axis(range(1, 25001))
target_valid2 = target_valid2.set_axis(range(1, 25001))
target_valid3 = target_valid3.set_axis(range(1, 25001))

predictions1 = pd.Series(predictions1)
predictions2 = pd.Series(predictions2)
predictions3 = pd.Series(predictions3)

predictions1 = predictions1.set_axis(range(1, 25001))
predictions2 = predictions2.set_axis(range(1, 25001))
predictions3 = predictions3.set_axis(range(1, 25001))

In [19]:
def profit(target, predictions):
    probs_sorted = predictions.sort_values(ascending=False)
    selected_df = target[probs_sorted.index][:200]
    total_profit = 450000 * selected_df.sum() - BUDGET
    return total_profit

state = np.random.RandomState(12345)

#target_valid1 = pd.Series(target_valid1)
#target_valid1 = pd.Series(target_valid1)
#target_valid1 = pd.Series(target_valid1)

predictions1 = pd.Series(predictions1)
predictions2 = pd.Series(predictions2)
predictions3 = pd.Series(predictions3)

values1 = [] 
for i in range(1000):
    target_subsample1 = target_valid1.sample(n=500, replace=True, random_state=state) 
    probs_subsample1 = predictions1[target_subsample1.index]
    values1.append(profit(target_subsample1, probs_subsample1))
    #print(target_subsample1)

values2 = [] 
for i in range(1000):
    target_subsample2 = target_valid2.sample(n=500, replace=True, random_state=state) 
    probs_subsample2 = predictions2[target_subsample2.index]
    values2.append(profit(target_subsample2, probs_subsample2))
    
values3 = [] 
for i in range(1000):
    target_subsample3 = target_valid3.sample(n=500, replace=True, random_state=state) 
    probs_subsample3 = predictions3[target_subsample3.index]
    values3.append(profit(target_subsample3, probs_subsample3))    
    
    
mean_profit1 = np.mean(values1 )
lower_bound1 = np.quantile(values1, 0.025)
upper_bound1 = np.quantile(values1, 0.975)

print("Средняя прибыль:", mean_profit1)
print("Нижняя граница:", lower_bound1)
print("Верхняя граница:", upper_bound1)
print()

mean_profit2 = np.mean(values2)
lower_bound2 = np.quantile(values2, 0.025)
upper_bound2 = np.quantile(values2, 0.975)

print("Средняя прибыль:", mean_profit2)
print("Нижняя граница:", lower_bound2)
print("Верхняя граница:", upper_bound2)
print()

mean_profit3 = np.mean(values3)
lower_bound3 = np.quantile(values3, 0.025)
upper_bound3 = np.quantile(values3, 0.975)

print("Средняя прибыль:", mean_profit3)
print("Нижняя граница:", lower_bound3)
print("Верхняя граница:", upper_bound3)
print()

Средняя прибыль: 425938526.91059244
Нижняя граница: -102090094.83793654
Верхняя граница: 947976353.3583689

Средняя прибыль: 518259493.6973249
Нижняя граница: 128123231.43308444
Верхняя граница: 953612982.0669085

Средняя прибыль: 420194005.3440501
Нижняя граница: -115852609.16001143
Верхняя граница: 989629939.8445739



In [20]:
loss_cases1 = sum([i < 0 for i in values1])
risk1 = loss_cases1 / len(values1)

loss_cases2 = sum([i < 0 for i in values2])
risk2 = loss_cases2 / len(values2)

loss_cases3 = sum([i < 0 for i in values3])
risk3 = loss_cases3 / len(values3)

print("Риск получить отрицательную прибыль для региона 1:", risk1)
print("Риск получить отрицательную прибыль для региона 2:", risk2)
print("Риск получить отрицательную прибыль для региона 3:", risk3)

Риск получить отрицательную прибыль для региона 1: 0.06
Риск получить отрицательную прибыль для региона 2: 0.003
Риск получить отрицательную прибыль для региона 3: 0.062


Согласно данным, лучше всего бурить скважину во втором регионе, потому что средняя прибыль в нем выше других регионов. Также риски, по сравнению с другими регионами, также невысокие. Таким образом можно сказать, что потенциально самый прибыльный регион -- регион №2. 

# Вывод

Таким образом в проекте было проанализировано 3 датасета, в которых встретились данные, описывающие месторождения. В рамках исследования были построены модели линейной регрессии с использованием библиотек машинног обучения, а также была проведена предобработка данных, в результате которой выяснилось, что мы потенацильно можем иметь дело с проблемой мультиколлинеарности. Также была посчитана потенциальная прибыль и выбран регион, в котором прибыль от бурения будет максимальной. 
    